#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2017


# Final Project - SocialRec

## Extract "Good" Data

In [4]:
# Add code here

## Biclustering

In [1]:
# Add code here

## Similarity

In [2]:
# Add code here

## Gradient Descent

In [36]:
# Add code here
# Add code here
import numpy
from collections import defaultdict;
import random;


numUsers = 10; # m
numItems = 20; # n
ku = kv = 5;


# Declare and fill the user-user similarity
userUserSimilarity = defaultdict(lambda: defaultdict(float));

for i in range(0, numUsers):
    for j in range(0, numUsers):
        userUserSimilarity[i][j] = 0.5;
        
        
# Declare and fill the user-user similarity
userItemSimilarity = defaultdict(lambda: defaultdict(float));

for i in range(0, numUsers):
    for j in range(0, numItems):
        userItemSimilarity[i][j] = 0.5;
        
# Important note!! This should only store values of 1.0 or 0.0
userRatedItemEncoding = defaultdict(lambda: defaultdict(float));

for i in range(0, numUsers):
    for j in range(0, numItems):
        rand = random.uniform(0.0, 1.0);
        
        # Randomly initialize this matrix.
        # In actuallity, this will be the correct values rather than random
        # TODO: Change to be correct
        if (rand > 0.5):
            userRatedItemEncoding[i][j] = 1.0;
        else:
            userRatedItemEncoding[i][j] = 0.0;

# We need to have the tag matrix
# numUsers x numItems x tagVector of user on item
userItemRatingsMatrix = defaultdict(lambda : defaultdict(float));
for i in range(0, numUsers):
    for j in range(0, numItems):
        rand = random.uniform(0.0, 1.0);
        
        # Randomly initialize this matrix.
        # In actuallity, this will be the correct values rather than random
        # TODO: Change to be correct
        if (rand > 0.5):
            userItemRatingsMatrix[i][j] = 1.0;
        else:
            userItemRatingsMatrix[i][j] = 0.0;

            
            
            
# This is a helper function
def FeatureTimesFeature(userFeatureVec, itemFeatureVec):
    userFeatureLen = len(userFeatureVec);
    itemFeatureLen = len(itemFeatureVec);
    
    if userFeatureLen != itemFeatureLen:
        print "Incorrect sized feature vecs, not equal";
        return nil;
    
    total = 0.0;
    for i in range(0, userFeatureLen):
        total = total + (userFeatureVec(i), itemFeatureVec(i))
    
    return total;

# This is a helper function
def FeatureMinusFeature(userFeatureOne, userFeatureTwo):
    oneLen = len(userFeatureOne);
    twoLen = len(userFeatureTwo);
    
    
    if userFeatureLen != itemFeatureLen:
        print "Incorrect sized feature vecs, not equal";
        return nil;
    
    result = defaultdict(float);
    
    total = 0.0;
    for i in range(0, userFeatureLen):
        result[i] = (userFeatureVec(i) - itemFeatureVec(i));
    
    return result;
            
# This is the objective function implementation

# This is dlds
def DLDS(i, userUserSim, userItemSim, userFeatureVectors, itemFeatureVectors, userTagsForItem):
    
    # n: num items
    # Sum from j = 1 to n
    term1 = 0.0;
    for j in range(j-1, numItems):
        term1 = term1 + I[i][j]*(S[i].Transposed()*V[j] - T[i][j])*V[j];
        
    # Sum over friends of user based on
    term2 = 0.0;
    
    
    # i and f are indices into users
    # j is an index into items
    for f in F[i]:
        term2 = term2 + UserItemSim(j, f) * (S[i] - S[f]);
        
    term2 = term2 * alpha;
    
    
    # i and f are indices into users
    # j is an index into items
    term3 = 0.0; 
    for f in F[i]:
        term3 = term3 + UserUserSimilarity(i, f) * (S[i] - S[f]);
    term3 = term3 * beta;
    
    term4 = 0.0;
    term4 = lambda1 * S[i];
    
    totalSum = totalSum + term1 + term2 + term3
    return totalSum;

# This is dldv
# Requires I, but lets make that global probably?

def DLDV(j, userUserSim, userItemSim, userFeatureVectors, itemFeatureVectors):
    
    # m is num users
    # Sum from i = 1 to m
    totalSum = 0.0;
    for i in range (0, numUsers):
        # Iij * (Si_Transpose * Vj - Tij)*Si 
        term1 = I[i][j] * (FeatureTimesFeature(userFeatureVectors[i], itemFeatureVectors[j]) - T[i][j]) * userFeatureVectors[i];
        # lambda_2*Vj
        term2 = lambda2 * itemFeatureVectors[i];
        
        totalSum = totalSum + term1 + term2;
    
    return totalSum;

In [37]:


# Important symbols to note
# U: The user list
# T: The user-item rating matrix
# Ri: The tag vector of user i
# F(i): The friend list of user i
# I the item list
# Si: The vector of user i
# R: the item-tag matrix
# Vk: The vector of item j


# Ok, here I implement the gradient decent algorithm for learning.
# This uses the previous box's learning algorithm

# R: 
# learningRate: how quickly convergence happens
# S: The similarities between users
# l: The correlations between users and items

# Ok, importatnt note from referenced paper
# R is an m x n rating matrix describing m users numerical ratings on n items


def RSboSN(R, learningRate, alpha, beta, maxN, userUserSimMatrix, userItemSimMatrix):
    
    # Initialize Sku and Vkv with random values
    # These are the calculated feature vectors
    # ku: the feature dimensions of user
    # kv: the feature dimensions of item
    
    # Dimensionality
    # Parameter set in the paper
    ku = 10;
    kv = 10;
    
    userFeatureVectors = defaultdict(lambda: defaultdict(float));
    itemFeatureVectors = defaultdict(lambda: defaultdict(float));
    
    # Randomize S
    for i in range(0, numUsers):
        for j in range(0, ku):
            userFeatureVectors[i][j] = random.uniform(0.01, 0.5);
    
    # Randomize V
    for i in range(0, numUsers):
        for j in range(0, kv):
            itemFeatureVectors[i][j] = random.uniform(0.01, 0.5);
    
    currentIteration = 0;
    
    while currentIteration < maxN:
        for i in range(0, numUsers):
            for j in range(0, numItems):
                Tij = T[i][j];
                
                # Calculate the derivitive of L with respect to S
                
                # For each u from 1 to ku
                for u in range(0, ku) :
                    dlds = 1.0;
                    userFeatureVectors[i][u] = userFeatureVectors[i][u] - (learningRate * dlds);
                
                # Calculate the derivitive of L with respect to S
                
                # For each u from 1 to ku
                for v in range(0, kv) :
                    dldv = 1.0;
                    itemFeatureVectors[i][v] = itemFeatureVectors[i][v] - (learningRate * dldv);
        
        currentIteration = currentIteration+1;
    
    print userFeatureVectors;
    print "\n\n";
    print itemFeatureVectors;

# We use:
# Num iterations = 1000
# Alpha = 0.01 From paper
# Beta = 0.01 From paper
# We use dimensionality = 80 from paper

R = defaultdict(float);
L = defaultdict(float);

RSboSN(R, 0.1, 0.01, 0.01, 1000, userUserSimilarity, userItemSimilarity);

defaultdict(<function <lambda> at 0x00000000063C69E8>, {0: defaultdict(<type 'float'>, {0: -1999.8502555988516, 1: -1999.6255477078773, 2: -1999.8171336223315, 3: -1999.8613605990147, 4: -1999.545212948173, 5: -1999.623109716304, 6: -1999.75049537341, 7: -1999.6049417945935, 8: -1999.5150320591572, 9: -1999.9031051130942}), 1: defaultdict(<type 'float'>, {0: -1999.699967594332, 1: -1999.8681212685617, 2: -1999.9287119422047, 3: -1999.8248208002015, 4: -1999.9058939528104, 5: -1999.6372806525937, 6: -1999.884765241389, 7: -1999.8084552467553, 8: -1999.5495199637128, 9: -1999.974720754194}), 2: defaultdict(<type 'float'>, {0: -1999.5154929679325, 1: -1999.73761987977, 2: -1999.6272205669525, 3: -1999.8741087092492, 4: -1999.6129846762096, 5: -1999.5084020695324, 6: -1999.7076419488296, 7: -1999.513489363671, 8: -1999.894535596626, 9: -1999.889928035794}), 3: defaultdict(<type 'float'>, {0: -1999.832206131495, 1: -1999.8556006281394, 2: -1999.725792077412, 3: -1999.8718132893832, 4: -1999